Project Title : DeliveryLogistics_Model

Prepared By : Rushikesh patil

In [3]:
# Import Data Manipulation Libraries
import numpy as np
import pandas as pd 

# Import Data Visiualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt

# Import Warnings
import warnings
warnings.filterwarnings(action='ignore')

# Import Scikit-Learn Libraries
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV,cross_val_score,KFold

# Import Machine Learning Model 
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV


In [72]:

# Function Defination 

# Step 1: Data Inegstion 

filepath = "https://raw.githubusercontent.com/rushikeshpatil-123/DeliveryLogistics_Model/refs/heads/main/data/raw/Delivery_Logistics.csv"

def data_ingestion():
    return pd.read_csv(filepath)

# Step 2: Data Exploration 

from collections import OrderedDict

def data_exploration(df):

    numerical_col = df.select_dtypes(exclude='object').columns
    categorical_col = df.select_dtypes(include='object').columns

    num_stats_list = []
    cat_stats_list = []

    # ------------------ Numerical Features ------------------
    for col in numerical_col:

        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1

        LW = Q1 - 1.5 * IQR
        UW = Q3 + 1.5 * IQR

        outlier_count = ((df[col] < LW) | (df[col] > UW)).sum()

        stats = OrderedDict({
            "Feature": col,
            "Mean": df[col].mean(),
            "Median": df[col].median(),
            "Maximum": df[col].max(),
            "Minimum": df[col].min(),
            "Q1": Q1,
            "Q3": Q3,
            "IQR": IQR,
            "Lower_Limit": LW,
            "Upper_Limit": UW,
            "Outlier_Count": outlier_count,
            "Skewness": df[col].skew(),
            "Kurtosis": df[col].kurt()
        })

        num_stats_list.append(stats)  
    numerical_stats_report = pd.DataFrame(num_stats_list)

    # ------------------ Categorical Features ------------------
    for col in categorical_col:
        cat_stats = OrderedDict({
            "Feature": col,
            "Unique Values": df[col].nunique(),
            "Mode": df[col].mode()[0],
            "Missing values": df[col].isnull().sum(),
            "Value Counts": df[col].value_counts().to_dict()
        })

        cat_stats_list.append(cat_stats)

    categorical_stats_report = pd.DataFrame(cat_stats_list)

    # ------------------ Dataset Info ------------------
    dataset_info = pd.DataFrame({
        "Feature": df.columns,
        "Dtype": df.dtypes.values,
        "Missing Values": df.isnull().sum().values,
        "Unique Values": df.nunique().values
    })

    return numerical_stats_report, categorical_stats_report, dataset_info


In [65]:
# Checking Descriptive Stats : Numerical Columns
numerical_stats_report


,Feature,Mean,Median,Maximum,Minimum,Q1,Q3,IQR,Lower_Limit,Upper_Limit,Outlier_Count,Skewness,Kurtosis
0,delivery_id,12500.500000,12500.500,24750.0100,250.9900,6250.75,18750.25,12499.50,-12498.500,37499.500,0,-1.066136e-16,-1.202673
1,distance_km,150.390436,151.000,297.1000,3.6000,75.90,224.90,149.00,-147.600,448.400,0,1.882955e-03,-1.196196
2,package_weight_kg,25.145898,25.145,49.5200,0.6700,12.68,37.66,24.98,-24.790,75.130,0,-2.606811e-03,-1.199496
3,delivery_rating,3.666000,4.000,5.0000,1.0000,3.00,5.00,2.00,0.000,8.000,0,-4.731261e-01,-0.738847
4,delivery_cost,864.944579,867.535,1632.7206,95.6674,490.80,1237.91,747.11,-629.865,2358.575,0,1.107873e-03,-1.172019


In [66]:
# Checking Descriptive Stats : Categorical columns
categorical_stats_report

,Feature,Unique Values,Mode,Missing values,Value Counts
0,delivery_partner,9,xpressbees,0,"{'xpressbees': 2826, 'fedex': 2818, 'dhl': 280..."
1,package_type,9,fragile items,0,"{'fragile items': 2848, 'pharmacy': 2810, 'doc..."
2,vehicle_type,6,ev bike,0,"{'ev bike': 4218, 'van': 4187, 'scooter': 4174..."
3,delivery_mode,4,two day,0,"{'two day': 6302, 'same day': 6279, 'express':..."
4,region,5,west,0,"{'west': 5095, 'central': 5060, 'south': 4977,..."
5,weather_condition,6,foggy,0,"{'foggy': 4219, 'stormy': 4198, 'rainy': 4171,..."
6,delivery_time_hours,20,1970-01-01 00:00:00.000000006,0,"{'1970-01-01 00:00:00.000000006': 3118, '1970-..."
7,expected_time_hours,9,1970-01-01 00:00:00.000000016,0,"{'1970-01-01 00:00:00.000000016': 6302, '1970-..."
8,delayed,2,no,0,"{'no': 18331, 'yes': 6669}"
9,delivery_status,3,delivered,0,"{'delivered': 18331, 'delayed': 5341, 'failed'..."


In [67]:
# Checking Dataset Information 
dataset_info

,Feature,Dtype,Missing Values,Unique Values
0,delivery_id,float64,0,24502
1,delivery_partner,object,0,9
2,package_type,object,0,9
3,vehicle_type,object,0,6
4,delivery_mode,object,0,4
5,region,object,0,5
6,weather_condition,object,0,6
7,distance_km,float64,0,2935
8,package_weight_kg,float64,0,4853
9,delivery_time_hours,object,0,20


# Exploratory Data Analysis

In [59]:
# Convert in to bins 
df['weight_category'] = pd.cut(
    df['package_weight_kg'],
    bins = [0,10,25,df['package_weight_kg'].max()],
    labels = [
        'Light (0 - 10 kg)',
        'Medium (10 - 25 kg)',
        'Heavy (25+ kg)'

    ],
    include_lowest = True
)

In [60]:
# Crosstab
def crosstab(df):
    crosstab1 = pd.crosstab(df['region'],df['weight_category'],margins=True)
    crosstab2 = pd.crosstab(df['vehicle_type'],df['weight_category'],margins=True)
    crosstab3 = pd.crosstab(df['package_type'],df['weight_category'],margins=True)
    crosstab4 = pd.crosstab(df['delivery_mode'],df['weight_category'],margins=True)
    crosstab5 = pd.crosstab(df['weather_condition'],df['weight_category'],margins=True)
    return crosstab1,crosstab2,crosstab3,crosstab4,crosstab5

In [ ]:
# Function Calling
crosstab1, crosstab2, crosstab3, crosstab4, crosstab5 = crosstab(df)

In [63]:
# Step 3: Data Preprocessing
def data_preprocessing(df):

    X = df.drop(columns= ['package_weight_kg'], axis =1 )
    y = df['package_weight_kg']

    # Split the Dataset into train and test

    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                        test_size=0.3,
                                                        random_state=0)
    
    # Use Encoding Techinque to convert all catergorical columns into numerical columns 

    from sklearn.preprocessing import LabelEncoder, MinMaxScaler
    categorical_col = X.select_dtypes(include = 'object').columns

    for i in  categorical_col:
        le = LabelEncoder()
        X_train[i] = le.fit_transform(X_train[i])
        X_test[i] = le.transform(X_test[i])

    # Using Normalization Technique

    sc = MinMaxScaler()
    X_train = sc.fit_transform(X_train)   # seen data
    X_test = sc.transform(X_test)    # unseen data 
    return X_train, X_test, y_train,y_test

In [6]:
# Step 4: Model Building 

def model_building(X_train, X_test, y_train, y_test):

    models = {
        "LinearRegression": LinearRegression(),
        "DecisionTreeRegressor":DecisionTreeRegressor(),
        "RandomForestRegressor":RandomForestRegressor(),
        "GradientBoostRegressor":GradientBoostingRegressor(),
        "AdaBoostRegressor":AdaBoostRegressor(),
        "SVR": SVR()
    }

    Regression_models = []
    
    for model_name, model in models.items():

        print(f"Training model: {model_name}")

        # Train
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)

        # Metrics
        r2_score_model = r2_score(y_test, y_pred)
        mse_model = mean_squared_error(y_test, y_pred)
        mae_model = mean_absolute_error(y_test, y_pred)

        Regression_models.append({
            "model name": model_name,
            "R2 Score": r2_score_model,
            "Mse": mse_model,
            "Mae": mae_model
        })

    
    Regression_models_report = pd.DataFrame(Regression_models)
    return Regression_models_report

In [71]:
model_report

,model name,R2 Score,Mse,Mae
0,LinearRegression,0.662078,69.456584,6.894695
1,DecisionTreeRegressor,0.950496,10.174986,2.298299
2,RandomForestRegressor,0.983866,3.316165,1.175722
3,GradientBoostRegressor,0.492508,104.310035,8.595420
4,AdaBoostRegressor,0.033770,198.599054,12.145630
5,SVR,0.225367,159.218107,10.872322


In [64]:
# Function Calling

df = data_ingestion()

numerical_stats_report, categorical_stats_report, dataset_info = data_exploration(df)

X_train, X_test, y_train, y_test = data_preprocessing(df)

model_report = model_building(X_train, X_test, y_train, y_test)

Training model: LinearRegression
Training model: DecisionTreeRegressor
Training model: RandomForestRegressor
Training model: GradientBoostRegressor
Training model: AdaBoostRegressor
Training model: SVR


In [12]:
# Use RandomForest with GridSearch CV

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Build Model with RandomForest 

rf = RandomForestRegressor(random_state=42)

# Hyperparameter grid

param_grid = {
    'n_estimators': [100],
    'max_depth':[None]

}

# GridSearch CV
grid = GridSearchCV(
    estimator = rf,
    param_grid = param_grid,
    cv = 5,
    n_jobs= -1,
    verbose= 1
)
# Fit On training Data 
grid.fit(X_train,y_train)

print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters: {'max_depth': None, 'n_estimators': 100}
Best score: 0.9799478341721878
